In [1]:
%matplotlib inline
from IPython.display import display
from IPython.display import Image
import sys
import math
import numpy as np
import argparse
import os
import shutil
import scipy
import re
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm, colors
from scipy import integrate
import scipy.constants as cst
import h5py
from itertools import product as itp
from pythtb import *

In [2]:
# # the path to read the input
# w90_input = w90('/Users/munenekariuki/Downloads/Simons_Foundation_Internship/Tight Binding Model/Tight Binding Model Examples/tb_analytic', 
#                 'sccuo2')
# # the cutoff Fermi energy
# fermi_ev = 9.6082


# # RED - our self-defined model where we set our zero energy, minimum hopping ampltide, and maximum distance we'd like to consider
# w90_model = w90_input.model(zero_energy=fermi_ev, min_hopping_norm=0.05, max_distance=None)
# (dist, ham) = w90_input.dist_hop()


# # corresponds to the BLACK line
# # Returns
# # - k-points reduced coordinates used in the interpolation in Wannier90 code
# # - energies interpolated by Wannier90 in eV.
# (w90_kpt, w90_evals) = w90_input.w90_bands_consistency()


# # corresponds to the red line
# int_evals = w90_model.solve_all(w90_kpt)

# # solve model
# # evals = w90_model.solve_all(w90_kpt)

# # fig, ax = plt.subplots(dpi=150)

# # for i in range(w90_evals.shape[0]):
# #     ax.plot(list(range(w90_evals.shape[1])), w90_evals[i]-fermi_ev, "k-", zorder=-100)

# # for i in range(int_evals.shape[0]):
# #     ax.plot(list(range(int_evals.shape[1])), int_evals[i], "r-", zorder=-50)

# # ax.set_xlim(0, int_evals.shape[1]-1)
# # ax.set_xlabel("K-path from Wannier90")
# # ax.set_ylabel("Band energy (eV)")

In [4]:
w90_model.display()

NameError: name 'w90_model' is not defined

In [18]:
# (fig, ax) = w90_model.visualize(0, 1)
# ax.set_title("Title goes here")
# fig.savefig("model.pdf")

In [19]:
print(w90_model.__dict__)

{'_dim_k': 3, '_dim_r': 3, '_lat': array([[ 1.76207011, -7.99610659,  0.        ],
       [ 1.76207011,  7.99610659,  0.        ],
       [ 0.        ,  0.        ,  3.93940453]]), '_orb': array([[-0.06720294,  1.06720298,  0.24999892],
       [-0.93605725,  0.93605725,  0.75000126]]), '_norb': 2, '_per': [0, 1, 2], '_nspin': 1, '_assume_position_operator_diagonal': False, '_nsta': 2, '_site_energies': array([ 0.018272, -0.130246]), '_site_energies_specified': array([ True,  True]), '_hoppings': [[(-0.501316+0j), 0, 0, array([0, 0, 1])], [(-0.452374+0j), 1, 1, array([0, 0, 1])], [(-0.062208+0j), 0, 0, array([0, 0, 2])], [(-0.064023+0j), 1, 1, array([0, 0, 2])], [(0.056434+0j), 0, 1, array([ 1,  0, -1])], [(0.056436+0j), 0, 1, array([1, 0, 0])]]}


In [20]:
orbs = np.matrix(w90_model._orb)
print(orbs)

[[-0.06720294  1.06720298  0.24999892]
 [-0.93605725  0.93605725  0.75000126]]


In [21]:
lat = np.matrix(w90_model._lat)
print(lat)

[[ 1.76207011 -7.99610659  0.        ]
 [ 1.76207011  7.99610659  0.        ]
 [ 0.          0.          3.93940453]]


In [22]:
print(w90_model._hoppings)

[[(-0.501316+0j), 0, 0, array([0, 0, 1])], [(-0.452374+0j), 1, 1, array([0, 0, 1])], [(-0.062208+0j), 0, 0, array([0, 0, 2])], [(-0.064023+0j), 1, 1, array([0, 0, 2])], [(0.056434+0j), 0, 1, array([ 1,  0, -1])], [(0.056436+0j), 0, 1, array([1, 0, 0])]]


In [23]:
# from mycolorpy import colorlist as mcp

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# print(lat.shape)

# from itertools import product as itp

# layers = 4
# len_itp = layers**3
# colors = mcp.gen_color(cmap="hsv",n=len_itp)

# for l, (l1, l2, l3) in enumerate(itp(range(layers), repeat=3)):
    
#     #print(l, l1, l2, l3)
#     for orb in orbs:
#         o = lat @ (orb.T + np.matrix([l1, l2, l3]).T)
#         ax.scatter(o[0], o[1], o[2], c=colors[l])
#         o = lat @ (orb.T - np.matrix([l1, l2, l3]).T)
#         ax.scatter(o[0], o[1], o[2], c=colors[l])

In [30]:
from triqs.lattice.tight_binding import TBLattice
import sympy as sp

def sympyfy(tb_lat_obj, analytical = True):
    r"""
    returns the analytical form of the momentum space hamiltonian of the tight-binding model 
    from a tight-binding lattice object, by utilizing Fourier series
    
    Parameters
    ----------
    tb_lat_obj: triqs TBLattice object
        triqs tight binding object
    analytical: boolean, default = True
        a boolean which will cause the function will return an analytical Hamiltonian, when true, and 
        an numerical Hamiltonian otherwise
    
    Returns
    -------
    Hk_numerical: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in numerical form
    Hk: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in reduced analytical or symbolic form
    """
    I = sp.I
    # matrix from the axis directions in momentum space
    kx, ky, kz = sp.symbols("kx ky kz")
    k_space_matrix = sp.Matrix([kx, ky, kz])

    # symbolic dot product representation between the lattice unit vectors
    # and the momentum space matrix
    a1k, a2k, a3k = sp.symbols("a1k a2k a3k")
    lattice = sp.Matrix([a1k, a2k, a3k])

    # the number of orbitals involved in the electron hoppings
    num_orb = tb_lat_obj.n_orbitals

    # dictionary containing details about the hopping of the electrons
    TB_lat_obj_hops = tb_lat_obj.hoppings 

    # maximum hopping distances of electrons in each of the axial directions
    max_x, max_y, max_z = list(np.max(np.array(list(TB_lat_obj_hops.keys())), axis = 0))

    # number of cells involved in the hopping of electrons in each of the axial directions
    num_cells_x, num_cells_y, num_cells_z = [2 * max_coord + 1 for max_coord in [max_x, max_y, max_z]]

    # basis of the 5D tensor real-space Hamiltonian
    Hrij = np.zeros((num_cells_x, num_cells_y, num_cells_z, num_orb, num_orb), dtype = sp.exp)

    # looping through the hopping parameters of the electrons involved in the inter-orbital hoppings
    # key represents the cell coordinates of where the electrons hop to relative to the home unit cell
    # hopping represents the matrix with the embedded hopping amplitudes
    for key, hopping in TB_lat_obj_hops.items():
        rx, ry, rz = key
        Hrij[rx + max_x, ry + max_y, rz + max_z] = hopping

    # basis of the exponential term in the calculation of Hk
    Hexp = np.empty_like(Hrij, dtype = sp.exp)

    # perform the Fourier transform
    for xi, yi, zi in itp(range(num_cells_x), range(num_cells_y), range(num_cells_z)):
        coefficients = np.array([xi - max_x, yi - max_y, zi - max_z])
        r = lattice.dot(coefficients)
        eikr = sp.exp(-I * r)
        Hexp[xi, yi, zi, :, :] = eikr
    
    # summation over all real space axes
    Hk = np.sum(Hrij * Hexp, axis = (0, 1, 2))
    
    # rewriting the exponential terms in the analytical expression in terms of 
    for i, j in itp(range(num_orb), repeat = 2):
        Hk[i, j] = Hk[i, j].rewrite(sp.cos)

    # dealing with the numerical Hamiltonian
    # we convert it to a SymPy matrix to use the substitutions method available in SymPy
    Hk_numerical = sp.Matrix(Hk)
    TB_lat_obj_units = tb_lat_obj.units
    TB_lat_obj_units_transpose = np.transpose(TB_lat_obj_units)
    
    # obtaining the unit vectors
    a1 = TB_lat_obj_units_transpose[0]
    a2 = TB_lat_obj_units_transpose[1]
    a3 = TB_lat_obj_units_transpose[2]

    # numerical dot products between the unit vectors
    # and the momentum space matrix
    a1k_numerical = a1.dot(k_space_matrix)[0]
    a2k_numerical = a2.dot(k_space_matrix)[0]
    a3k_numerical = a3.dot(k_space_matrix)[0]
    
    # performing the numerical dot product substitutions
    Hk_numerical = Hk_numerical.subs(a1k, a1k_numerical)
    Hk_numerical = Hk_numerical.subs(a2k, a2k_numerical)
    Hk_numerical = Hk_numerical.subs(a3k, a3k_numerical)

    # converting the numerical Hamiltonian to a NumPy array from a SymPy matrix
    Hk_numerical = np.array(Hk_numerical)
    
    # returning the analytical or numerical form of the Hamiltonian
    # depending on the user's preference
    if analytical:
        return Hk
    return Hk_numerical

In [43]:
from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('sccuo2 Files', 'sccuo2')
fermi_ev = 9.6082
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.05, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)
print("The analytical expression \n", sympyfy(w90_triqs))
# print("The numerical expression \n", sympyfy(w90_triqs, False))
from sympy import Matrix, exp, I, cos, Symbol
def check_exponential(matrix):
    for sublist in matrix:
        for element in sublist:
            if element.has(exp):
                return True
    return False

x = Symbol('x')
y = Symbol('y')
exampleMatrix = Matrix([
    [exp(x), cos(x), 1],
    [exp(y), cos(y), 2]
])

exampleMatrixAsNumpyArray = np.array(exampleMatrix)
print(check_exponential(exampleMatrixAsNumpyArray))

# print(check_exponential(sympyfy(w90_triqs)))
# print(type(sympyfy(w90_triqs)[0][0]))

The analytical expression 
 [[-1.002632*cos(a3k) - 0.124416*cos(2*a3k) + 0.0182719999999996
  -0.056436*I*sin(a1k) - 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)]
 [0.056436*I*sin(a1k) + 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)
  -0.904748*cos(a3k) - 0.128046*cos(2*a3k) - 0.130246]]
True


In [17]:
from triqs.lattice.tight_binding import TBLattice
import sympy as sp

e0 = 0.0183
e1 = -0.1302

t1 = -0.501316
t2 = -0.452374

t3 = -0.062208
t4 = -0.064023

t5 = 0.056434
t6 = 0.056436

hop = {
        (0, 0, 0)   : [[ e0, 0], [0, e1]],
        (0, 0, 1)   : [[ t1, 0], [0, t2]],
        (0, 0, -1)  : [[ t1, 0], [0, t2]],
        (0, 0, 2)   : [[ t3, 0], [0, t4]],
        (0, 0, -2)  : [[ t3, 0], [0, t4]],
        (1, 0, -1)  : [[ 0, t5], [0, 0]],
        (-1, 0, 1)  : [[ 0, 0], [t5, 0]],
        (-1, 0, 0)  : [[ 0, 0], [t6, 0]],
        (1, 0, 0)   : [[ 0, t6], [0, 0]]
      }


L = TBLattice(units = [(1.7621, -7.9961, 0.) , (1.7621, 7.9961, 0.), (0., 0., 3.9394)], hoppings = hop,                     
              orbital_positions = [(-0.0672, 1.0672, 0.25), (-0.9361, 0.9361, 0.75)])


I = sp.I
kx, ky, kz = sp.symbols("kx ky kz")
k = sp.Matrix([kx, ky, kz])
norb = 2

#fourier transform the hoppings to find the 8x8 noninteracting H(k) in the Nambu site basis
mx = 1 #max hopping distance in x direction                                        
my = 1 #max hopping distance in y direction                                        
mz = 2 #max hopping distance in y direction                                        
nx = 2*mx+1                                                                        
ny = 2*my+1
nz = 2*mz+1

Hrij = np.zeros((nx, ny, nz, norb, norb), dtype = np.complex_)
for r in L.hopping_dict():
    rxind = r[0] + mx
    ryind = r[1] + my
    rzind = r[2] + mz
    Hrij[rxind, ryind, rzind] = L.hopping_dict()[r]

Hexp = np.empty_like(Hrij, dtype = sp.exp)
for xi in range(nx):
    for yi in range(ny):
        for zi in range(nz):
            r = np.array([xi-mx, yi-my, zi-mz])
            r = lat.T.dot(r)
            eikr = sp.exp(-I*k.dot(r))
            Hexp[xi, yi, zi, :, :] = eikr
Hkrij = Hrij * Hexp
Hk = np.sum(Hkrij, axis = (0,1,2))
for i, j in itp(range(norb), repeat=2):
    Hk[i, j] = Hk[i, j].rewrite(sp.cos).simplify()
print(Hk)

[[-1.002632*cos(3.939404532*kz) - 0.124416*cos(7.878809064*kz) + 0.0183
  0.056436*exp(I*(-1.762070113*kx + 7.996106593*ky)) + 0.056434*exp(I*(-1.762070113*kx + 7.996106593*ky + 3.939404532*kz))]
 [0.056436*exp(I*(1.762070113*kx - 7.996106593*ky)) + 0.056434*exp(I*(1.762070113*kx - 7.996106593*ky - 3.939404532*kz))
  -0.904748*cos(3.939404532*kz) - 0.128046*cos(7.878809064*kz) - 0.1302]]


In [13]:
Ks = 2*np.pi*np.linalg.inv(lat)
print(lat)
print(Ks)

subs = 0

with open("/mnt/ceph/users/sbeck/materials/srcacuo/sccuo2/wan_conv/sccuo2_band.kpt", 'r') as A:
    lines = A.readlines()[1:]

ks = []
for k in range(len(lines)):
    l = lines[k].split()
    ks.append([float(l[0]), float(l[1]), float(l[2])])
    
# path=np.matrix([[0.0,0.0,0.0],[0.0,0.0,0.5],[0.25,0.25,0.5],[0.25,0.25,0.0],[0.0, 0.0, 0.0],[-0.25,0.25,0.0]])
# ks = []
# nk = 25
# for k in range(len(path)-1):
#     p = path[k+1] - path[k]
#     for i in range(nk):
#         ks.append([path[k, 0]+p[0, 0]/nk*i, path[k, 1]+p[0, 1]/nk*i, path[k, 2]+p[0, 2]/nk*i])
#         # print(ks[-1])
# ks.append([path[-1, 0], path[-1, 1], path[-1, 2]])
# # print(ks[-1])

# Hk2 = [[], [], []]
# for ki, k in enumerate(ks):
#     new_k = Ks @ np.matrix(k).T
#     Hk2_tmp = 0*Hk
#     for i, j in itp(range(norb), repeat=2):
#         Hk2_tmp[i, j] = Hk[i, j].evalf(5, subs={"kx":new_k[0, 0], "ky":new_k[1, 0], "kz":new_k[2, 0]})
#     eigs = np.linalg.eigvals(Hk2_tmp.astype(complex))
#     Hk2[0].append(ki)
#     Hk2[1].append(eigs[0])
#     Hk2[2].append(eigs[1])

a = 3.52414
b = 15.99221
c = 3.93940

e1 = 0.0183
e2 = -0.1302

t11 = -0.501316
t22 = -0.452374

tp11 = -0.062208
tp22 = -0.064023

t12 = 0.056434

Hk2 = [[], [], []]
for ki, k in enumerate(ks):
    new_k = Ks @ np.matrix(k).T
    
    # print(new_k. T)
    
    Hk2_tmp = 0*Hk
    
    Hk2_tmp[0, 0] = 2*t11*np.cos(c*new_k[2]) + 2*tp11*np.cos(2*c*new_k[2]) + e1
    # Hk2_tmp[0, 1] = t12*np.exp(-1j*(a/2*new_k[0]-b/2*new_k[1]))*(1+np.exp( 1j*c*new_k[2]))
    # Hk2_tmp[1, 0] = t12*np.exp( 1j*(a/2*new_k[0]-b/2*new_k[1]))*(1+np.exp(-1j*c*new_k[2]))
    Hk2_tmp[1, 1] = 2*t22*np.cos(c*new_k[2]) + 2*tp22*np.cos(2*c*new_k[2]) + e2
    
    # Hk2_tmp[0, 1] = t12*(1+np.exp( 1j*c*new_k[2]))
    # Hk2_tmp[1, 0] = t12*(1+np.exp(-1j*c*new_k[2]))
    
    Hk2_tmp[0, 1] = t12*(1+np.exp( 1j*c*new_k[2]))
    Hk2_tmp[1, 0] = 2*t12*np.cos(c*new_k[2]/2)
    
    eigs = np.linalg.eigvals(Hk2_tmp.astype(complex))
    Hk2[0].append(ki)
    Hk2[1].append(eigs[0])
    Hk2[2].append(eigs[1])
    
plt.plot(Hk2[0], Hk2[1])
plt.plot(Hk2[0], Hk2[2])



[[ 1.76207011 -7.99610659  0.        ]
 [ 1.76207011  7.99610659  0.        ]
 [ 0.          0.          3.93940453]]
[[ 1.78289878  1.78289878  0.        ]
 [-0.39289029  0.39289029  0.        ]
 [ 0.          0.          1.59495814]]


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/ceph/users/sbeck/materials/srcacuo/sccuo2/wan_conv/sccuo2_band.kpt'

In [14]:
w90_input = w90('/mnt/ceph/users/sbeck/materials/srcacuo/sccuo2/wan_conv/', 'sccuo2')
fermi_ev = 9.6082

w90_model = w90_input.model(zero_energy=fermi_ev, min_hopping_norm=0.05, max_distance=None)
# w90_model = w90_input.model(zero_energy=fermi_ev, min_hopping_norm=0.0, max_distance=3.8586)
# w90_model = w90_input.model(zero_energy=fermi_ev)
(dist, ham) = w90_input.dist_hop()
(w90_kpt, w90_evals) = w90_input.w90_bands_consistency()
int_evals = w90_model.solve_all(w90_kpt)

# solve model
evals = w90_model.solve_all(w90_kpt)

fig, ax = plt.subplots(dpi=150)

for i in range(w90_evals.shape[0]):
    ax.plot(list(range(w90_evals.shape[1])), w90_evals[i]-fermi_ev, "k-", zorder=-100)

for i in range(int_evals.shape[0]):
    ax.plot(list(range(int_evals.shape[1])), int_evals[i], "r-", zorder=-50)

ax.plot(Hk2[0], Hk2[1], color="blue", linestyle=':')
ax.plot(Hk2[0], Hk2[2], color="blue", linestyle=':')
    
ax.set_xlim(0, int_evals.shape[1]-1)
ax.set_xlabel("K-path from Wannier90")
ax.set_ylabel("Band energy (eV)")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/ceph/users/sbeck/materials/srcacuo/sccuo2/wan_conv//sccuo2.win'